# The Bellman subnetwork

## Introduction to optimization and operations research.

Michel Bierlaire


In [ ]:

import itertools
from typing import Any

import numpy as np
from IPython.core.display_functions import display
from matplotlib import pyplot as plt
from networkx import DiGraph
from teaching_optimization.networks import draw_network
from teaching_optimization.networks.shortest_path_algorithm import ShortestPathAlgorithm


The objective of this exercise is to construct Bellman subnetworks from the optimal labels of the
shortest path algorithm.

Bellman's equation is defined as

$$\lambda_j = \min_{(i,j)\in \mathcal{A}} (\lambda_i + c_{ij}).$$

Bellman's subnetwork consists in selecting for each node except the origin one arc that verifies
Bellman's equation.

If every cycle in the the_network has positive length, that is, if there is no cycle of cost zero, each
Bellman subnetwork is a graph called the *shortest path graph*.
We will try below to enumerate all Bellman subnetworks.

In [ ]:

positions = {
    'a': (0, 0),
    'b': (2, 1.5),
    'c': (2, -1.5),
    'd': (4, 0),
}

nodes = list(positions.keys())

arcs = [
    ('a', 'b', 1),
    ('a', 'c', 1),
    ('b', 'd', 1),
    ('c', 'd', 1),
    ('b', 'c', 0),
]

first_network: DiGraph = DiGraph()
for node in nodes:
    first_network.add_node(node, pos=positions[node])
first_network.add_weighted_edges_from(arcs, weight='cost')
fig, ax = plt.subplots(figsize=(8, 6))
draw_network(the_network=first_network, attr_edge_labels='cost', ax=ax)
plt.show()


We first solve the shortest path problem.

Initialization

In [ ]:
the_algorithm = ShortestPathAlgorithm(
    the_network=first_network, the_cost_name='cost', the_origin='a'
)


Run the algorithm.  If the output is `True`, it managed to find the optimal labels.

In [ ]:
the_algorithm.shortest_path_algorithm()


Optimal labels

In [ ]:
display(the_algorithm.labels)



We now write a function that identifies the Bellman arcs for a given node.
When comparing two real numbers, it is better to use the `numpy`function `isclose` instead
of the standard `==` operator.

In [ ]:


def bellman_arcs(
    a_network: DiGraph, a_node: Any, labels: dict[Any, float], cost_name: str
) -> list[tuple[Any, Any]]:
    """Identifies the Bellman arcs of a node

    :param a_network: the_network object
    :param a_node: the node under interest
    :param labels: the optimal labels
    :param cost_name: name of the cost attribute
    :return: a list of arcs
    """
    optimal_arcs = [
        (upstream, a_node)
        for upstream, _ in a_network.in_edges(a_node)
        if np.isclose(
            labels[upstream] + a_network[upstream][a_node][cost_name],
            labels[a_node],
        )
    ]
    return optimal_arcs



We identify the Bellman arcs for each node
The expected result is as follows:

In [ ]:
expected_result = {
    'a': [],
    'b': [('a', 'b')],
    'c': [('a', 'c'), ('b', 'c')],
    'd': [('b', 'd'), ('c', 'd')],
}
display(expected_result)


In [ ]:
the_bellman_arcs = {
    node: bellman_arcs(
        a_network=first_network,
        a_node=node,
        labels=the_algorithm.labels,
        cost_name='cost',
    )
    for node in first_network.nodes
}
display(the_bellman_arcs)


Let's print the result.

In [ ]:
for node, arcs in the_bellman_arcs.items():
    print(f'Bellman arcs for node {node}: {arcs}')


In order to plot the Bellman subnetworks, we need to consider all possible combinations
of Bellman arcs.

In [ ]:
all_lists = [a_list for a_list in the_bellman_arcs.values() if a_list]
all_combinations = list(itertools.product(*all_lists))
for one_instance in all_combinations:
    print(one_instance)


Let's plot all Bellman's subnetworks. ALl of them have the same set of nodes, and a different set
of Bellman arcs.

In [ ]:
for one_instance in all_combinations:
    # Create a directed graph
    a_graph: DiGraph = DiGraph()

    # Add nodes to the graph
    a_graph.add_nodes_from(first_network.nodes(data=True))

    # Add arcs to the graph
    a_graph.add_edges_from(one_instance)

    fig, ax = plt.subplots(figsize=(8, 6))
    draw_network(the_network=a_graph, ax=ax)
    plt.show()



# Second example

We investigate the Bellman subnetworks for another example.

In [ ]:

positions = {
    'a': (0, 1),
    'b': (1, 2),
    'c': (1, 0),
    'f': (2, 0),
    'd': (2, 2),
    'e': (3, 1),
    'h': (4, 0),
    'g': (4, 2),
    'i': (5, 1),
}

nodes = list(positions.keys())

arcs = [
    ('a', 'b', 10),
    ('a', 'c', 12),
    ('b', 'd', -12),
    ('b', 'f', 4),
    ('c', 'd', 7),
    ('c', 'b', 8),
    ('c', 'f', 6),
    ('d', 'g', 16),
    ('d', 'e', -3),
    ('f', 'd', 7),
    ('e', 'g', 7),
    ('e', 'i', -1),
    ('e', 'h', 6),
    ('e', 'f', -4),
    ('f', 'h', 15),
    ('g', 'i', 8),
    ('h', 'i', 5),
]

second_network: DiGraph = DiGraph()
for node in nodes:
    second_network.add_node(node, pos=positions[node])
second_network.add_weighted_edges_from(arcs, weight='cost')
fig, ax = plt.subplots(figsize=(8, 6))
draw_network(the_network=second_network, attr_edge_labels='cost', ax=ax)
plt.show()


We solve the shortest path problem.

Initialization

In [ ]:
the_algorithm = ShortestPathAlgorithm(
    the_network=second_network, the_cost_name='cost', the_origin='a'
)


Run the algorithm.  If the output is `True`, it managed to find the optimal labels.

In [ ]:
the_algorithm.shortest_path_algorithm()


Optimal labels

In [ ]:
display(the_algorithm.labels)



We identify the Bellman arcs for each node

In [ ]:
the_bellman_arcs = {
    node: bellman_arcs(
        a_network=second_network,
        a_node=node,
        labels=the_algorithm.labels,
        cost_name='cost',
    )
    for node in second_network.nodes
}


In [ ]:
for node, arcs in the_bellman_arcs.items():
    print(f'Bellman arcs for node {node}: {arcs}')


Consider all possible combinations

In [ ]:
all_lists = [a_list for a_list in the_bellman_arcs.values() if a_list]
all_combinations = list(itertools.product(*all_lists))
for one_instance in all_combinations:
    print(one_instance)


Let's plot all Bellman's subnetworks

In [ ]:
for one_instance in all_combinations:
    # Create a directed graph
    a_graph: DiGraph = DiGraph()

    # Add nodes to the graph
    a_graph.add_nodes_from(second_network.nodes(data=True))

    # Add arcs to the graph
    a_graph.add_edges_from(one_instance)

    fig, ax = plt.subplots(figsize=(8, 6))
    draw_network(the_network=a_graph, ax=ax)
    plt.show()


Note that the last one is not a graph. This may happen if the the_network contains a cycle with zero cost,
which is the case of the cycle `d -> e -> f -> d`